In [28]:
# @hidden_cell
# INSERT PROJECT TOKEN HERE



In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='39a65707-7291-457c-890a-5d0da4be86a3', project_access_token='p-2+AwN/GnfgY5oC6fQatKfPnw==;g/tHxvT6Gr1Fa4YewGjYnw==:o4n0HL6YLUSHlt+gUdMNvXq+rNMUU6Wzbgy2+yd/n+8H26jKDQPZGRq5h/kpBT9L5N6fqDcluk8iM9KHC85uevc0xDRyOF3ytw==')
pc = project.project_context

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space({'token':'p-2+AwN/GnfgY5oC6fQatKfPnw==;g/tHxvT6Gr1Fa4YewGjYnw==:o4n0HL6YLUSHlt+gUdMNvXq+rNMUU6Wzbgy2+yd/n+8H26jKDQPZGRq5h/kpBT9L5N6fqDcluk8iM9KHC85uevc0xDRyOF3ytw=='})



# Lab 3: Intro to Building Prompt Templates with LangChain

### If you have troubles with your local installation you can use this notebook instead. 

Please note that compared to the local version the set-up and the way the model is accessed and credentiasl are read are slightly different. 

Welcome to the Lab 3. 

In the previous lab, we explored the challenges of prompt engineering; learning how to tweak our wording, choose different models, plus optimize model parameters. Minor changes can significantly enhance the results generated by language models.

In this lab, we will apply our new knowledge to a real-world use case as we continue learning about best practices related to prompt coding. Using the [Watson Machine learning Python SDK](https://ibm.github.io/watson-machine-learning-sdk/) to programmatically interact with watsonx.ai, we will use prompt templating techniques provided by the [LangChain Python library](https://python.langchain.com/) to streamline our interactions with the language model and maximize its potential.

The concept of Prompt Templates provided by LangChain allows you to construct prompt templates that can be easily filled with specific information to generate a wide range of outputs that you can provide to watsonx.ai. You can even leverage prompt templates specific to few-shot prompting, as you will see below.

## Recreating Prompt Builder Prompts Using LangChain Prompt Patterns

### Scenario: Personalized Recommendation for XYZ Retail Company <a id="step3"></a>

XYZ Retail is a popular online retail store that sells a wide range of products, including electronics, clothing, home goods, and more. They have a large customer base and want to provide a personalized shopping experience to enhance customer satisfaction and boost sales.

To achieve that goal, XYZ wants to leverage generative AI to create fact sheets about each of their customers. These fact sheets will summarize relevant information such as customer demographics (name, age, location), and purchase history. These fact sheets will help XYZ Retail's sales team build stronger customer relationships, increase customer satisfaction and drive repeat purchases.


You start by performing prompt engineering in Prompt Lab, and you might test base model output with an initial prompt like this:

![title](./images/prompt_without_example.png)

The model's recommendation is not accurate or useful as the customer Michael Jones had bought toys and games not outdoor activewear. Fortunately you learned in the Prompt Engineering lab that Few Shot Learning can help you obtain better results. 

What happens when we provide a few examples using Prompt Builder to guide the LLM into generating more meaningful recommendations. 

![title](./images/prompt_with_example.png)


Great, the product recommendation for Michael Jones is much better.  However how do you productionize your few shot prompting to generate recommendations for all of XYZ Retail customers? Copy and pasting each customer's info into Prompt Builder would take too long.  

You'll need a programmatic solution.  Maybe you could even generate a large set of examples then use that for Tuning a model in watsonx.ai.  But we're getting ahead of ourselves as you'll learn about building a Prompt Tuning dataset in a later lab.

## 1. Load the required libraries  <a id="step1"></a>

In [2]:
# due to some Python version issue together with Langchain with the Python version available on Cloud we cannot use standard LangChain installation and must go through generative ai extension instead at the time being
# !pip install langchain
# !pip install typing-extensions
!pip install "ibm-generative-ai[langchain]"

In [2]:
import os

import pandas as pd
from langchain import PromptTemplate, FewShotPromptTemplate
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
import os, getpass
import requests
from ibm_cloud_sdk_core import IAMTokenManager

# Setup credentials

In the next steps, you need the credentials mentioned above, especially your IBM Cloud API key and IBM Cloud regional URL (eg, https://us-south.ml.cloud.ibm.com)

# Enter WML endpoint URL

In [4]:
endpoint_url = input("Please enter your WML endpoint url (hit enter): ")

Please enter your WML endpoint url (hit enter): https://us-south.ml.cloud.ibm.com


# Enter API Key

In [5]:
access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your IBM Cloud api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

Please enter your IBM Cloud api key (hit enter): ········


# Enter Project ID

Since we are working in a Cloud project, you should need to add anything here as it will be read from the notebook environment

In [6]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

# Helper function to set up model endpoint

In [7]:
# Helper Function - slightly different than local helper function
class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = f"{endpoint_url}/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]
        else:
            return response.text

## 2. Create a Factsheet for each customer using Prompt Patterns  <a id="step2"></a>

### **2.1 What is a Prompt Template?**

The [PromptTemplate class](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) in the [LangChain Python library](https://python.langchain.com/docs/get_started/introduction) provides a flexible approach to creating prompts from structured templates.  We will use the PrompTemplate class to simplify creation of our few shot prompts for XYZ Retail.

XYZ Retail has provided you their customer's data in .csv format. To generate prompts for each customer, you will need to transform the prompt that you engineered in Prompt Builder into a more useful programmatic format. Using the PromptTemplate class, you can easily substitute customer data to generate one or multiple prompts.

The PromptTemplate class defines a schema where variables to replace are placed inside curly braces "{}". In Python parlance, it's simply using "f-strings" under the hood. These curly braces serve as a placeholder for the actual data that will be substituted into the template.

Let's see how this works in practice.

### **2.2 Creating a simple prompt from a template**

A prompt template can be created using the PromptTemplate class from a string or .txt file. There are [additional PromptTemplate examples](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.prompt.PromptTemplate.html#langchain.prompts.prompt.PromptTemplate) provided in the LangChain documentation.

#### 2.2.1 Prompt Template From String

In [8]:
# template is a string with variable names in curly brackets
pattern = "input: {name} {family_name} is {age} and lives in {location}. They bought {purchase_history}"

# generate template
prompt_template = PromptTemplate.from_template(pattern)
prompt_template.template

'input: {name} {family_name} is {age} and lives in {location}. They bought {purchase_history}'

In [9]:
# now let's provide some values and generate our prompt
# notice how the variables coincide with those we specified in curly brackets
prompt = prompt_template.format(name="Jane", 
                                family_name="Doe",
                                age=43,
                                location="San Francisco, CA",
                                purchase_history = "groceries, household goods and travel supplies")

prompt

'input: Jane Doe is 43 and lives in San Francisco, CA. They bought groceries, household goods and travel supplies'

#### 2.2.2 Prompt Template From File
Prompt patterns can also be stored as a txt file:

In [10]:
# Get the url
_path_to_file = project.get_file("customer_factsheet_lang.txt").read().decode()

In [11]:
# We create a template from a file:
# this time we provide the variable names in a list
# due to watson studio file reading options we need to change this a bit compared to the local file and not use the PromptTemplate.from_file

# LOCAL
# example_prompt = PromptTemplate.from_file(_path_to_file,
#                                 input_variables=["name", "family_name", "age","city", "state", 
#                                                  "purchase_history", "recommendation_1", "recommendation_2"])

# CLOUD
example_prompt = PromptTemplate.from_template(_path_to_file)
                                # input_variables=["name", "family_name", "age","city", "state", 
                                #                  "purchase_history", "recommendation_1", "recommendation_2"])


print(example_prompt.template)

input: "{name} {family_name} is {age} years old and lives in {city}, {state}. Their purchase history includes {purchase_history}."
output: "Recommendations:\n Item 1: {recommendation_1}\nItem 2: {recommendation_2}"


Just like in 2.1, we can populate this teamplate from a dictionary containing the values of the input variables. Looping over three examples:

In [12]:
# we can iterate through a list to populate the template

examples = [
    {
        "name":"Jane", 
        "family_name":"Doe", 
        "age":43, 
        "city":"San Francisco", 
        "state":"CA",
        "purchase_history":"groceries, household goods and travel supplies", 
        "recommendation_1":"Basket of organic fruits",
        "recommendation_2":"Lightweight carry-on suitcase"
    },{
        "name":"Siamak", 
        "family_name":"Baharoo", 
        "age":57, 
        "city":"Chicago", 
        "state":"IL",
        "purchase_history":"Books electronics home_goods", 
        "recommendation_1":"Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It has a glare-free display and a long battery life, so you can read for hours on end without having to worry about running out of power.",
        "recommendation_2": "Google Home Mini - This smart speaker is perfect for controlling your home's smart devices with your voice. You can use it to play music, set alarms, get news, and more. It's also a great way to stay connected with friends and family."
    },{
        "name":"Luis", 
        "family_name":"Cooli", 
        "age":21, 
        "city":"New York City", 
        "state":"NY",
        "purchase_history":"Clothing shoes cosmetics", 
        "recommendation_1":"Aritzia Wilfred Free Sweater - This soft and cozy sweater is perfect for a casual day out. It's available in a variety of colors, so you can find the perfect one to match your style.",
        "recommendation_2":"Steve Madden Pointed Toe Pumps - These stylish pumps are perfect for a night out on the town. They're comfortable and versatile, so you can wear them with a variety of outfits."
    }
]

for example in examples: 
    print(example_prompt.format(**example))

input: "Jane Doe is 43 years old and lives in San Francisco, CA. Their purchase history includes groceries, household goods and travel supplies."
output: "Recommendations:\n Item 1: Basket of organic fruits\nItem 2: Lightweight carry-on suitcase"
input: "Siamak Baharoo is 57 years old and lives in Chicago, IL. Their purchase history includes Books electronics home_goods."
output: "Recommendations:\n Item 1: Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It has a glare-free display and a long battery life, so you can read for hours on end without having to worry about running out of power.\nItem 2: Google Home Mini - This smart speaker is perfect for controlling your home's smart devices with your voice. You can use it to play music, set alarms, get news, and more. It's also a great way to stay connected with friends and family."
input: "Luis Cooli is 21 years old and lives in New York City, NY. T

## 3. Create Prompt Examples based on Customers Factsheet <a id="step3"></a>
The value of PromptTemplate arises when generating a large number of prompts either as examples for bulk evaluation of an engineered prompt or for creation of a Tuning dataset

### 3.1 Create a few shot prompt
We will start by creating a FewShotPromptTemplate object. This class allows to create a prompt made of few repetitions of a PromptTemplate. Details can be found in the [FewShotPromptTemplate class documentation](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples)

In [13]:
# Next step create a few shot prompt template

few_shot_examples = examples[:2]
few_shot_input = examples[2].copy()
del few_shot_input['recommendation_1']
del few_shot_input['recommendation_2']

def make_few_shot_prompt(few_shot_examples, few_shot_input):
    """
    Generate a few-shot prompt using the FewShotPromptTemplate class.

    Parameters:
    - few_shot_examples: List of examples to be shown as few-shot examples.
    - few_shot_input: Input for which the prompt will be generated.

    Returns:
    - A string representing the formatted few-shot prompt.
    """
    prompt = FewShotPromptTemplate(
        examples=few_shot_examples, 
        example_prompt=example_prompt, 
        suffix='input: "{name} {family_name} is {age} years old and lives in {city}, {state}. Their purchase history includes {purchase_history}."\noutput: ', 
        input_variables=["name", "family_name", "age", "city","state", "purchase_history"]
    )
    # Return the formatted prompt using the provided input data
    return prompt.format(**few_shot_input)


few_shot_prompt = make_few_shot_prompt(few_shot_examples, few_shot_input)
print(few_shot_prompt)

input: "Jane Doe is 43 years old and lives in San Francisco, CA. Their purchase history includes groceries, household goods and travel supplies."
output: "Recommendations:\n Item 1: Basket of organic fruits\nItem 2: Lightweight carry-on suitcase"

input: "Siamak Baharoo is 57 years old and lives in Chicago, IL. Their purchase history includes Books electronics home_goods."
output: "Recommendations:\n Item 1: Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It has a glare-free display and a long battery life, so you can read for hours on end without having to worry about running out of power.\nItem 2: Google Home Mini - This smart speaker is perfect for controlling your home's smart devices with your voice. You can use it to play music, set alarms, get news, and more. It's also a great way to stay connected with friends and family."

input: "Luis Cooli is 21 years old and lives in New York City, NY.

### 3.2 Bulk Creation of Prompts
Using the FewShotPromptTemplate class, we can now create a function that generates a list a few shot prompts populating them iteratively from values directy extracted from a notebook.

We can choose how many single prompts are include in one few shot prompt. The output of the function is a list of few shot prompts. 

### Insert 'customer_factsheet.csv' here: 

In [14]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='X-0-jEu7L98aubTRDwQ46G0AXTXSy-0JRx9zHu5gWklE',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'testmby-donotdelete-pr-vwhokyenmfxj0y'
object_key = 'customer_factsheet.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,name,family_name,age,city,state,purchase_history,recommendation_1,recommendation_2
0,John,Smith,30,San Francisco,CA,Books electronics home_goods,Kindle Paperwhite - This e-reader is perfect f...,Google Home Mini - This smart speaker is perfe...
1,Jane,Doe,25,New York,NY,Clothing shoes cosmetics,Aritzia Wilfred Free Sweater - This soft and c...,Steve Madden Pointed Toe Pumps - These stylish...
2,Michael,Jones,40,Seattle,WA,Toys games sporting_goods,IKEA Ektorp Sofa - This comfortable and stylis...,Samsung 4K Smart TV - This high-definition TV ...
3,Sarah,Johnson,50,Chicago,IL,Furniture appliances home_improvement_supplies,IKEA KALLAX Shelving Unit - This versatile she...,LG WM3900HWA Washer and Dryer Set - This washe...
4,David,Williams,60,Atlanta,GA,Groceries household_items travel_items,Basket of organic fruits,Lightweight carry-on suitcase


In [15]:
# Specify the path to the CSV file containing the data - local only
# csv_file_path = "./data/customer_factsheet.csv"

def sub_all_from_csv(df_name, n_prompt_examples=2):
    """
    Generates a list of few-shot prompts using the FewShotPromptTemplate class. 
    The prompts are populated iteratively from values extracted from a CSV file.

    Parameters:
    - csv_file_path: The path to the CSV file.
    - n_prompt_examples: The number of examples included in one few-shot prompt.

    Returns:
    - list_of_prompts: A list of few-shot prompts.
    """

    # df = pd.read_csv(csv_file_path)
    examples = [example for _, example in df_name.transpose().to_dict().items()]

    i=0
    list_of_prompts = []

    while i < len(df_name):
        few_shot_examples = examples[i:i+n_prompt_examples]
        few_shot_input = examples[i+n_prompt_examples].copy()
        del few_shot_input['recommendation_1']
        del few_shot_input['recommendation_2']

        list_of_prompts.append(make_few_shot_prompt(few_shot_examples, few_shot_input))
        
        i = i+n_prompt_examples +1

    # Return the list of few-shot prompts
    return list_of_prompts

In [16]:
list_of_prompts = sub_all_from_csv(df_data_1)
print(list_of_prompts[0])

input: "John Smith is 30 years old and lives in San Francisco, CA. Their purchase history includes Books electronics home_goods."
output: "Recommendations:\n Item 1: Kindle Paperwhite - This e-reader is perfect for book lovers who want a lightweight and portable device that can hold thousands of books. It has a glare-free display and a long battery life, so you can read for hours on end without having to worry about running out of power.\nItem 2: Google Home Mini - This smart speaker is perfect for controlling your home's smart devices with your voice. You can use it to play music, set alarms, get news, and more. It's also a great way to stay connected with friends and family."

input: "Jane Doe is 25 years old and lives in New York, NY. Their purchase history includes Clothing shoes cosmetics."
output: "Recommendations:\n Item 1: Aritzia Wilfred Free Sweater - This soft and cozy sweater is perfect for a casual day out. It's available in a variety of colors, so you can find the perfect

### 3.2 Additional Examples
You can explore [additional examples using the PromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.prompt.PromptTemplate.html#langchain.prompts.prompt.PromptTemplate)

## 4. Prompt evaluation and few shot learning from bulk created prompts <a id="step4"></a>
In the prior examples, you created a "2-shot learning" prompt.  I.e. there were three inputs but only two complete outputs.  By using a larger dataset this way, you can perform bulk testing of your prompt.

E.g. two of your data samples are used to train before generating the "output" of the 3rd. You can now execute these few shot prompts to see how well our engineered prompt works across numerous examples

### 4.1 Import Watsonx.ai access credentials and load model
Make sure you copied the .env file that you created earlier into the same directory as this notebook

In [17]:
# Different than local version

# Parameters
model_params = {
         "decoding_method": "greedy",
         "min_new_tokens": 50,
         "max_new_tokens": 100
}

# Model ID
model_id="google/flan-ul2"

In [18]:
# instantiate with access token and project ID from within Cloud project
prompt = Prompt(access_token, project_id)

### 4.2 Send prompts to Watsonx.ai

In [28]:
# responses = [model.generate_text(prompt) for prompt in list_of_prompts]
responses = [prompt.generate(prompt_template_text, model_id, model_params)['generated_text'] for prompt_template_text in list_of_prompts]


for i, response in enumerate(responses):
    lines = str(list_of_prompts[i]).strip().split("\n")
    #user_description = str(lines[8])  # TODO check hard-coded number
    #print(f"\n{user_description}")
    print(f"\n MODEL OUTPUT: {i} , {response}")


 MODEL OUTPUT: 0 , "Recommendations:n Item 1: X-Box One - The X-Box One is the latest in the X-Box line of gaming systems. It features a high definition 1080p screen, a high-powered GPU, and a high-performance CPU. It also has Kinect, which allows you to play games by simply moving your body.nItem 2: X-Box 360 - The X

 MODEL OUTPUT: 1 , "Recommendations:n Item 1: Makeup - MAC Ruby Woo LipsticknItem 2: Skincare - Clinique Dramatically Different MoisturizernItem 3: Fashion - MAC x Patrick Starrr CollectionnItem 4: Location - Los Angeles, CAn

 MODEL OUTPUT: 2 , "Recommendations:n Item 1: RefrigeratornItem 2: FurniturenItem 3: AppliancesnItem 4: Home Improvement SuppliesnnItem 5: HomenItem 6: FurniturenItem 7: AppliancesnItem 8: HomenItem 9: Home Improvement SuppliesnItem 10: HomenItem 11: FurniturenItem 12: Home


### Few shot prompt analysis
These results are not bad.  An X-Box for a customer with a history of buying toys and games.  Likewise cosmetics and furniture for the other two customers accurately reflects their purchase history.  

## 5. Congratulations
Congratulations on completing the lab and exploring the fascinating world of bulk creation of Few Shot Prompts using PromptTemplate! 

Through the practical use case of generating personalized product recommendations, you have witnessed the power of tailoring prompts to individual customer profiles. By incorporating customer-specific details and programmatically generating bulk examples, you can fine-tune the model for your specific use case, resulting in more accurate and tailored outputs. 

Continuously iterating and refining your prompts based on these examples will unlock the full potential of language models and enhance their performance across various domains. Keep experimenting and leveraging prompt engineering techniques to optimize your interactions with language models and drive impactful results in your projects.